### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 3 sobre los servicios web de la EMT creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga las siguientes páginas:

*  Una página principal que mostrará un conjunto de varios enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * Servicio 1:Servicio de obtener la ruta optima entre dos destinos usando los servicios de la EMT
   
   * Servicio 2:Servicio de mostrar los lugares de interés entre dos destinos indicados.
   
   * Servicio 3:Servicio de mostrar las paradas de autobús más cercanas con toda la información acerca de la parada: líneas que pasan, frecuencia, ...
   
* Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la descripción de la ruta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

* Cuando el usuario pulsa sobre el servicio 2 se le mostrará un formulario en el que podrá introducir el nombre de la calle y número de un origen y un destino de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará un listado con los lugares de interés recuperados en la ruta entre las dos calles. De cada resultado mostrará la información disponible: nombre, teléfono, calle,...En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial.[3 puntos]

* Este servicio es nuevo, y requiere el uso de otro servicio web de la EMT. En este caso, se va a utilizar el servicio GetStreet de GEO. Cuando el usuario pulsa sobre el servicio 3 se le mostrará un formulario en el que podrá introducir el nombre de una calle y un número de la capital de Madrid, y cuando pulse sobre un botón de "Enviar", se le mostrará una nueva página que mostrará la información de las paradas más cercanas a la calle con toda la información acerca de las líneas de autobuses que paran en esas paradas. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial. [3 puntos]

## Normas de entrega

* Fecha tope de entrega: 22/11/2017
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [ ]:
from bottle import route, default_app, template, run, static_file, error,request
from lxml import etree
from xml.etree import ElementTree
from xml.etree.ElementTree import Element
import requests
#Practica 3
#####################
#-----------
#EJERCICIO 2
def coordenadas(calle,numero):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'description': calle,
        'streetNumber': numero,
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }

    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoCalle.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoCalle.xml", "rt")
    
    #Comprobamos si se han encontrado la calle introducida
    #Si el servidor web nos devuelve un fichero XML que contenga más de una línea, la habrá encontrado
    #Si solo contiene una línea, no la ha econtrado
    numLineas=0
    for line in f:
        numLineas+=1
        if(numLineas > 1): #Si contiene más de una línea paramos el bucle porque ya sabemos que el resultado es bueno
            break
    if(numLineas < 2): #Si no tiene al menos 2 líneas, no ha encontrado la calle y salimos de la función devolviendo un -1
        return -1 #Esto lo hacemos con vistas a los ejercicios 4 y 5 para indicar que la calle no existe
    
    else: #Si el fichero tiene al menos 2 líneas, ha encontrado la calle y continuamos la ejecución
        arbol=ElementTree.parse(f)
        nodo = arbol.find('Site')
        coorX = nodo.find('CoordinateX')
        coorY = nodo.find('CoordinateY')
        xy=[]
        xy.append(coorX.text)
        xy.append(coorY.text)

        return xy

    
#-----------    
#EJERCICIO 3
def caminoOptimo(xyOrigen,xyDestino,listaMostrar):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': xyOrigen[0],
        'coordinateYFrom':xyOrigen[1],
        'originName':'',
        'coordinateXTo': xyDestino[0],
        'coordinateYTo':xyDestino[1],
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoRuta.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoRuta.xml", "rt")
    arbol=ElementTree.parse(f)
    nodo = arbol.find('ListRouteData')
    nodo2= nodo.find('RouteData')
    nodo3=nodo2.find('DescriptionRouteData')

    fecha = nodo3.find('DescriptionDate')
    horaInicio = nodo3.find('DescriptionInitTime')
    horaFin = nodo3.find('DescriptionEstimateTimeArrival')
    trasbordos = nodo3.find('Transfers')
    duracion = nodo3.find('LongJourney')

    #quitamos lo que no valga en el texto
    fecha=fecha.text.replace("Fecha:","")
    horaInicio=horaInicio.text.replace("Hora de Salida:","")
    #NUEVO
    listaMostrar.append(fecha)
    listaMostrar.append(horaInicio)
    listaMostrar.append(horaFin.text)
    listaMostrar.append(trasbordos.text)
    listaMostrar.append(duracion.text)
   
    listaSecciones=nodo2.find('ListSectionRoute')
    #Recorro todas las secciones
    for seccion in listaSecciones.iter("Section"):
        secciones= list(seccion)
        for paso in secciones:
            if paso.tag == "WalkingLeg":
                leer= paso.find('SourceWalkingLeg')
                leer2=leer.find('RouteDescription')
                #si es lo ultimo que hace quitamos el " hasta"
                andar=leer2.text.replace(" hasta","")
                listaMostrar.append(andar)
            elif paso.tag == "BusLeg":
                leer= paso.find('SourceBusLeg')
                leer2=leer.find('RouteDescription')
                listaMostrar.append(leer2.text)

#-----------
#EJERCICIO 4
#Función auxiliar que utilizaremos para los ejercicios 4 y 5 que pide 2 calles al usuario y comprueba que sean validas
def origenDestino(cOrigen,nOrigen, cDestino,nDestino):
    xyOrigen=coordenadas(cOrigen,nOrigen)
    xyDestino=coordenadas(cDestino,nDestino)
    
    return xyOrigen,xyDestino

def calcularRutaMasOptima(calleOrigen,ncalleOrigen,calleDestino,ncalleDestino,listaMostrar):
    xyOrigen,xyDestino = origenDestino(calleOrigen,ncalleOrigen,calleDestino,ncalleDestino)  
    caminoOptimo(xyOrigen,xyDestino,listaMostrar)
    
#-----------                

####################
@route('/')
def index():
    return template("index.tpl")
@route('/cogerCalles')
def cogerCalles():
    return template("cogerCalles.tpl")
@route('/cogerCalles',method='POST') 
def do_cogerCalles():
    calleOrigen = request.forms.get('calleOrigen')
    ncalleOrigen = request.forms.get('ncalleOrigen')
    calleDestino = request.forms.get('calleDestino')
    ncalleDestino = request.forms.get('ncalleDestino')
    listaMostrar=[]#Aqui guardo todos los print que hacia en la practica 3 para enviarlo al template y que este los muestre
    listaMostrar.append(calleOrigen)
    listaMostrar.append(ncalleOrigen)
    listaMostrar.append(calleDestino)
    listaMostrar.append(ncalleDestino)
    calcularRutaMasOptima(calleOrigen,ncalleOrigen,calleDestino,ncalleDestino,listaMostrar)
    return template("rutaOptima.tpl",listaMostrar=listaMostrar)
@error(404)
def error404(error):
    return 'Error en la página solicitada'

run(host="localhost", port=8080, debug=True)